In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, precision_score, recall_score
warnings.filterwarnings('ignore')

Датасет использовал раньше в курсовых, решил взять его. Т.к уже знаю данные таргет и зависимости

In [2]:

df_train = pd.read_csv('data/Credit_default.csv')

In [3]:
df_train.isna().sum()

Home Ownership                     0
Annual Income                   1557
Years in current job             371
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    4081
Bankruptcies                      14
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                    1557
Credit Default                     0
dtype: int64

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Home Ownership                7500 non-null   object 
 1   Annual Income                 5943 non-null   float64
 2   Years in current job          7129 non-null   object 
 3   Tax Liens                     7500 non-null   float64
 4   Number of Open Accounts       7500 non-null   float64
 5   Years of Credit History       7500 non-null   float64
 6   Maximum Open Credit           7500 non-null   float64
 7   Number of Credit Problems     7500 non-null   float64
 8   Months since last delinquent  3419 non-null   float64
 9   Bankruptcies                  7486 non-null   float64
 10  Purpose                       7500 non-null   object 
 11  Term                          7500 non-null   object 
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

In [5]:
#Обработка пропусков
def fill_na(df):
    #Делаем новые признаки метки для пропусков
    df[['Annual Income_nan', 'Years in current job_nan', 'Months since last delinquent_nan', 'Credit Score_nan']] = 0
    #Делаем переменную коэффициент для умножения признака и заполнения пропуска
    corr_coef_mean = df_train['Annual Income'].mean()/df_train['Monthly Debt'].mean()
    #Заполняем Annual Income и делаем признак метку = 1
    df['Annual Income_nan'][df['Annual Income'].isna()] = 1
    df['Annual Income'][df['Annual Income'].isna()] = df['Monthly Debt'][df['Annual Income'].isna()] * corr_coef_mean
    #Заполняем Years in current job как не работающий и делаем признак метку = 1
    df['Years in current job_nan'][df['Years in current job'].isna()] = 1
    df['Years in current job'][df['Years in current job'].isna()] = "don't work"
    #Заполняем Months since last delinquent нулями и признак метку единицами
    df['Months since last delinquent_nan'][df['Months since last delinquent'].isna()] = 1
    df['Months since last delinquent'][df['Months since last delinquent'].isna()] = 0
    #Заполняем пропуски Bankruptcies delinquent
    df['Bankruptcies'][df['Bankruptcies'].isna()] = df['Number of Credit Problems'][df['Bankruptcies'].isna()]
    #Заполняем пропуски Credit Score медианой (пока) и признак метку = 1
    df['Credit Score_nan'][df['Credit Score'].isna()] = 1
    df['Credit Score'][df['Credit Score'].isna()] = df['Credit Score'].median()
    return df

df_train = fill_na(df_train)



In [6]:
def fix_outliers(df):
    df[['Maximum Open Credit_outlier']] = 0
    df['Maximum Open Credit_outlier'][df['Maximum Open Credit'] > 30000000] = 1
    df['Maximum Open Credit'][df['Maximum Open Credit'] > 30000000] = 30000000
    return df
df_train = fix_outliers(df_train)


In [7]:
#Делаем feature encoding а именно ранги категориальных признаков
#При необходимости базовые признаки от них можно потом удалить

def create_rank_cat(df):
    #Создаем и инциализируем новые признаки
    df[['Home Ownership_rank', 'Years in current job_rank', 'Tax Liens_rank', 'Number of Credit Problems_rank', 'Bankruptcies_rank', 'Term_rank', 'Purpose_rank', 'No problems']] = 0
    #Заполняем признак No problems
    df['No problems'][(df['Tax Liens'] == 0) & (df['Bankruptcies'] == 0) & (df['Number of Credit Problems'] == 0)] = 1
    #Исправляем ошибочное значение признака или однозначное в одно значение
    df['Home Ownership'][df['Home Ownership'] == 'Have Mortgage'] = 'Home Mortgage'
    #Делаем ранги для домовладений
    df['Home Ownership_rank'][df['Home Ownership'] == 'Home Mortgage'] = 1
    df['Home Ownership_rank'][df['Home Ownership'] == 'Own Home'] = 2
    #Делаем ранги для времени работы
    df['Years in current job_rank'][df['Years in current job'].isin(['< 1 year'])] = 1
    df['Years in current job_rank'][df['Years in current job'].isin(['1 year', '2 years', '3 years'])] = 2
    df['Years in current job_rank'][df['Years in current job'].isin(['4 years', '5 years'])] = 3
    df['Years in current job_rank'][df['Years in current job'].isin(['6 years', '7 years', '8 years', '9 years', '10+ years'])] = 4
    #Делаем ранги для Tax Liens
    df['Tax Liens_rank'][df['Tax Liens'].isin([i for i in range(1, 4)])] = 1
    df['Tax Liens_rank'][df['Tax Liens'] == 0] = 2
    #Делаем ранги для Number of Credit Problems
    df['Number of Credit Problems_rank'][df['Number of Credit Problems'].isin([2, 3])] = 1
    df['Number of Credit Problems_rank'][df['Number of Credit Problems'] == 1] = 2
    df['Number of Credit Problems_rank'][df['Number of Credit Problems'] == 0] = 3
    #Заполняем ранг для банкротства
    df['Bankruptcies_rank'][df['Bankruptcies'] > 0] = 1
    #Заполняем ранг Term
    df['Term_rank'][df['Term'] == 'Long Term'] = 1
    #Заполняем ранг Purpose
    df['Purpose_rank'][df['Purpose'].isin(['buy house', 'buy a car', 'home improvements', 'medical bills'])] = 1
    df['Purpose_rank'][df['Purpose'].isin(['business loan', 'small business'])] = 2
    df['Purpose_rank'][df['Purpose'] == 'debt consolidation'] = 3
    df[['Tax Liens', 'Number of Credit Problems', 'Bankruptcies']] = \
    df[['Tax Liens', 'Number of Credit Problems', 'Bankruptcies']].astype('int64')
    return df

df_train = create_rank_cat(df_train)


In [8]:
#После обработки датасета, обозначаем таргет, и убираем категориальные переменные, т.к кодирование уже сделали
target = pd.Series(df_train['Credit Default'])
features = df_train.drop(['Credit Default', 'Home Ownership', 'Years in current job', 'Purpose', 'Term'], axis=1)
#Бьем на трейн тест
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    shuffle=True,
                                                    test_size=0.30,
                                                    random_state=42)

In [9]:
df_train['Credit Default'].value_counts()

0    5387
1    2113
Name: Credit Default, dtype: int64

In [10]:
forest = RandomForestClassifier(max_depth=3, random_state=42, class_weight={0: 1, 1: 2})
forest.fit(X_train, y_train)
preds = forest.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')
roc_forest = roc_auc_score(y_test, preds)


Best Threshold=0.43203767430571427, F-Score=0.547, Precision=0.512, Recall=0.588


In [11]:
mod_data = df_train.copy()

# get the indices of the positives samples
pos_ind = np.where(mod_data['Credit Default'].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)
# leave just 50 of the positives marked
perc = 0.50
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]


Using 1057/2113 as positives and unlabeling the rest


In [12]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())



target variable:
 -1    6443
 1    1057
Name: class_test, dtype: int64


In [13]:
mod_data = mod_data.sample(frac=1)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)



(1057, 31) (1057, 31)


In [14]:
pu_forest = RandomForestClassifier(max_depth=3, random_state=42)
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

pu_forest.fit(sample_train.drop(columns=['class_test', 'Credit Default', 'Home Ownership', 'Years in current job', 'Purpose', 'Term']),
          sample_train['class_test'])

y_predict = pu_forest.predict(sample_test.drop(columns=['class_test', 'Credit Default', 'Home Ownership', 'Years in current job', 'Purpose', 'Term']))
f1_pu = f1_score(sample_test['Credit Default'], y_predict)
roc_pu = roc_auc_score(sample_test['Credit Default'], y_predict)
recall_pu = recall_score(sample_test['Credit Default'], y_predict, average='binary')
precision_pu = precision_score(sample_test['Credit Default'], y_predict, average='binary')

In [15]:
metrics = pd.DataFrame({'classic_forest': [precision[ix], recall[ix], fscore[ix], roc_forest], 'PU_forest': [precision_pu, recall_pu, f1_pu, roc_pu]}, index=['precision', 'recall', 'f1', 'roc_auc'])
metrics


,classic_forest,PU_forest
precision,0.511905,0.345291
recall,0.588146,0.514477
f1,0.547383,0.413238
roc_auc,0.759397,0.659645
